In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
target_list = pd.read_csv('./target_list.csv')

In [3]:
target_list.head()

,symbol,url
0,NWSA,https://www.marketbeat.com/earnings/transcript...
1,STE,https://www.marketbeat.com/earnings/transcript...
2,CRL,https://www.marketbeat.com/earnings/transcript...
3,DIS,https://www.marketbeat.com/earnings/transcript...
4,OXY,https://www.marketbeat.com/earnings/transcript...


In [4]:
# target_list

In [5]:
def fetch_data(symbol, url):
    print("Fetching stock {} via {}".format(symbol, url))
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    content = soup.select("div.transcript-discussion")

    report = content[0]
    qa_session = content[1]

    report_conversation = report.select("div.flex-shrink-1.position-relative.transcript-arrow")
    # print(len(report_conversation), "found in the report session")

    report_transcript = []
    for i in range(len(report_conversation)):
        speaker_content = report_conversation[i].select("div.transcript-line-speaker")[0]
        speaker_info = [child.text.strip() for child in  speaker_content.children]
        speaker_name = speaker_info[0]
        speaker_title = speaker_info[1] if len(speaker_info) >1 else speaker_info[0]
        for index, item in enumerate(report_conversation[i].select("p")):
            data = [speaker_name, speaker_title, item.text]
            report_transcript.append(data)

    report_df = pd.DataFrame(report_transcript, columns=['Speaker','Title','Content'])
    report_df.to_csv('./earning_reports/'+ symbol +'-{}-report.tsv'.format(url.split('/')[-2]), sep='\t', index=False)    

    qa_conversation = qa_session.select("div.flex-shrink-1.position-relative.transcript-arrow")
    # print(len(qa_conversation), "found in the Q&A session")

    qa_transcript = []
    for i in range(len(qa_conversation)):
        speaker_content = qa_conversation[i].select("div.transcript-line-speaker")[0]
        speaker_info = [child.text.strip() for child in  speaker_content.children]
        speaker_name = speaker_info[0]
        speaker_title = speaker_info[1] if len(speaker_info) >1 else speaker_info[0]
        for index, item in enumerate(qa_conversation[i].select("p")):
            data = [speaker_name, speaker_title, item.text]
            qa_transcript.append(data)

    qa_df = pd.DataFrame(qa_transcript, columns=['Speaker','Title','Content'])
    qa_df.to_csv('./earning_reports/'+ symbol +'-{}-qa.tsv'.format(url.split('/')[-2]), sep='\t', index=False)

    full_df = pd.concat([report_df, qa_df], axis=0)
    full_df = full_df.reset_index().drop(columns='index')
    full_df.to_csv('./earning_reports/'+ symbol +'-{}-full.tsv'.format(url.split('/')[-2]), sep='\t', index=False)
    
    print("Done getting stock {} via {}".format(symbol, url))
    

In [6]:
for index, row in target_list.iterrows():
    print(index)
    fetch_data(row['symbol'], row['url'])

0
Fetching stock NWSA via https://www.marketbeat.com/earnings/transcripts/92009/
Done getting stock NWSA via https://www.marketbeat.com/earnings/transcripts/92009/
1
Fetching stock STE via https://www.marketbeat.com/earnings/transcripts/91374/
Done getting stock STE via https://www.marketbeat.com/earnings/transcripts/91374/
2
Fetching stock CRL via https://www.marketbeat.com/earnings/transcripts/90177/
Done getting stock CRL via https://www.marketbeat.com/earnings/transcripts/90177/
3
Fetching stock DIS via https://www.marketbeat.com/earnings/transcripts/89740/
Done getting stock DIS via https://www.marketbeat.com/earnings/transcripts/89740/
4
Fetching stock OXY via https://www.marketbeat.com/earnings/transcripts/90019/
Done getting stock OXY via https://www.marketbeat.com/earnings/transcripts/90019/
5
Fetching stock CE via https://www.marketbeat.com/earnings/transcripts/89734/
Done getting stock CE via https://www.marketbeat.com/earnings/transcripts/89734/
6
Fetching stock ICE via htt

In [11]:
target_list.shape

(100, 2)